# The Algorithm

In [1]:
import sympy
import random
import math

Step 1: get $h(x,y)$ from $f(x,y)$ and $g(x,y)$, remembering that $h(x,y) = x(g\partial_xf - f\partial_xg) - y(g\partial_yf - g\partial_yg)$

In [2]:
x = sympy.Symbol('x')
y = sympy.Symbol('y')
def hgetter(f,g):
    return x.as_poly()*(g*sympy.diff(f, x) - f*sympy.diff(g, x)) - y.as_poly()*(g*sympy.diff(f, y) - f*sympy.diff(g, y))

In [3]:
f = sympy.Poly('x**2 + 2*x*y + y**2')
g = sympy.Poly('x**3 + y')

In [37]:
p = sympy.Poly('x**(2/3) + y**(1/3)')
print(p)

Poly(x**(1/3)**2 + y**(1/3), x**(1/3), y**(1/3), domain='ZZ')


In [42]:
dir(p)

['EC',
 'EM',
 'ET',
 'LC',
 'LM',
 'LT',
 'TC',
 '__abs__',
 '__add__',
 '__bool__',
 '__call__',
 '__class__',
 '__complex__',
 '__delattr__',
 '__dir__',
 '__div__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getnewargs__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__int__',
 '__le__',
 '__long__',
 '__lt__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rdiv__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rmod__',
 '__rmul__',
 '__rpow__',
 '__rsub__',
 '__rtruediv__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '_args',
 '_assumptions',
 '_compare_pretty',
 '_diff_wrt',
 '_eval_adjoint',
 '_eval_as_leading_term',
 '_eval_conjugate',
 '_eval_derivative',
 '_eval_diff',
 '_eval_evalf',
 '_eva

In [4]:
h = hgetter(f,g)
print(h)

Poly(-x**5 - 6*x**4*y - 5*x**3*y**2 + 3*x**2*y + 2*x*y**2 - y**3, x, y, domain='ZZ')


Auxiliar functions

In [5]:
def monicMaker(h):
    '''
    This function peforms a rotation on a polynomial of two variables
    h(x,y). That is, it maps h(x,y) to h(x+ny, -nx + y)) for some n
    such that the resulting polynomial is monic in y.
    
    TO-DO:
        - implement a smarter choosing of n, perhaps considering the
          roots of the polynomial.
    '''
    new_h = h
    while sympy.LC(new_h, y) not in sympy.CC:
        n = random.randint(1,10)
        new_h = h.as_expr().subs([(x, x + n*y), (y, -n*x + y)], simultaneous=True)
    new_h = new_h * (1/sympy.LC(new_h, y))
    return new_h.as_poly()

In [6]:
h = sympy.Poly(x**2*y**2 + 2*x*y + 4, x, y)
print(monicMaker(h))
print(sympy.LC(monicMaker(h), y) == 1)

Poly(x**4 + 15/2*x**3*y + 193/16*x**2*y**2 - 1/2*x**2 - 15/2*x*y**3 - 15/8*x*y + y**4 + 1/2*y**2 + 1/4, x, y, domain='QQ')
True


In [7]:
def newtonAutomorphism(h, q, p):
    '''
    This function performs the Newton Automorphism (x maps to x ** q
    and y maps to y(x ** p)) to a polynomial h, returning a sympy expr
    (not a polynomial). Here, p and q are expected to be rational
    numbers.
    '''
    return h.as_expr().subs([(x, x ** q), (y, y*x**p)], simultaneous=True)

In [8]:
newtonAutomorphism(h, 1/2, 1)
print(h)

Poly(x**2*y**2 + 2*x*y + 4, x, y, domain='ZZ')


In [45]:
def componentsOfy(F):
    '''
    This function returns a list with the homogenous
    components of h as a polynomial in y in increasing order.
    '''
    x = sympy.Symbol('x')
    y = sympy.Symbol('y')
    list_of_Fprimes = [F]
    list_of_fs = [F.eval(y, 0)]
    for j in range(1, sympy.degree(F, y)+1):
        nextFprime = sympy.diff(list_of_Fprimes[j-1], y)
        list_of_Fprimes.append(nextFprime)
        list_of_fs.append((1/math.factorial(j))*nextFprime.eval(y, 0).as_poly(x))
    return list_of_fs

def componentsOfx(F):
    '''
    This function returns a list with the homogenous
    components of h as a polynomial in y in increasing order.
    '''
    x = sympy.Symbol('x')
    y = sympy.Symbol('y')
    list_of_Fprimes = [F]
    list_of_fs = [F.eval(x, 0)]
    for j in range(1, sympy.degree(F, x)+1):
        nextFprime = sympy.diff(list_of_Fprimes[j-1], x)
        list_of_Fprimes.append(nextFprime)
        list_of_fs.append((1/math.factorial(j))*nextFprime.eval(x, 0).as_poly(y))
    return list_of_fs

def phiAutomorphism(h):
    '''
    This function performs the phi automorphism to a polynomial that's monic in y,
    so that the term that's with degree(h, y) - 1 banishes. Also returns the 
    term that was substracted for the construction of the inverse.
    '''
    x = sympy.Symbol('x')
    y = sympy.Symbol('y')
    list_of_bs = componentsOfy(h)
    b1 = list_of_bs[-2]
    #print(sympy.degree(h, y))
    #print(h)
    #print(list_of_bs)
    return substituteInPoly(h, [(y, y- (1/sympy.degree(h, y))*b1.as_expr())]), b1

In [29]:
def substituteInPoly(poly, list_of_subs):
    return poly.as_expr().subs(list_of_subs, simultaneous=True).as_poly()

In [12]:
def leastDegreeInx(b):
    x = sympy.Symbol('x')
    y = sympy.Symbol('y')
    k = 0
    while(sympy.simplify(b/x**k).subs(x, 0) == 0):
        k += 1
    return k

In [13]:
def urgetter(h):
    '''
    Given a polynomial 
        h(x,y) = y ** d + b_2(x)y**(d-2) + ... + b_d(x)
    this function returns the tuple r, u_r where u_r is the
    degree of b_r such that u_r/r is minimal.
    '''
    list_of_bs = componentsOfy(h)
    list_of_bs = list_of_bs[::-1] # To follow the convention of the proof.
    list_of_least_degrees = [leastDegreeInx(b) for b in list_of_bs[2:]] #Here, position 0 is really from b2.
    list_of_quotients = [k/(i+2) for (i, k) in enumerate(list_of_least_degrees)]
    r_indexer = list_of_quotients.index(min(list_of_quotients))
    return r_indexer + 2, list_of_least_degrees[r_indexer]

## Testing

In [14]:
x = sympy.Symbol('x')
y = sympy.Symbol('y')

In [31]:
f = sympy.Poly(x, x, y)
g = sympy.Poly(y, x, y)

h= hgetter(f,g)
print(h)
print(h == 2*x*y)

Poly(2*x*y, x, y, domain='ZZ')
True


In [34]:
print(substituteInPoly(h, [(x, y), (y, x)]))

Poly(2*x*y, x, y, domain='ZZ')


In [16]:
f = sympy.Poly(x**2 + y**2)
g = sympy.Poly(2*y**3 + 4*x**2*y)

h= hgetter(f,g)
print(h)

Poly(4*x**4*y - 2*x**2*y**3 + 2*y**5, x, y, domain='ZZ')


hgetter is working perfectly

In [17]:
h = sympy.Poly(2*x*y)
monicMaker(h)

Poly(-x**2 - 35/6*x*y + y**2, x, y, domain='QQ')

monicMaker too, let's try the automorphisms

In [18]:
h1 = monicMaker(h)
print(h1)
print(h1.as_poly(y))
h1.as_poly(y).LC()

Poly(-x**2 + y**2, x, y, domain='ZZ')
Poly(y**2 - x**2, y, domain='ZZ[x]')


1

phi is working well too. Let's try Newton's.

In [19]:
h = sympy.Poly(2*x*y)
newtonAutomorphism(h, sympy.Rational(1,2), sympy.Rational(1,3))

2*x**(5/6)*y

also working well, we need to be wary of the way rationals are entered.

In [46]:
h = sympy.Poly(x**2*y**2 + (2*x**3 + x + 1)*y + x**5)
monic_h = monicMaker(h)
# print(componentsOfy(monic_h))
# print(monic_h)
modified_h, inverser = phiAutomorphism(monic_h)
print(modified_h, inverser)
print(modified_h)
print(componentsOfy(modified_h))
(r, ur) = urgetter(modified_h)
print(r, ur)

Poly(1.73472347597681e-18*x**4*y - 4.33680868994202e-19*x**4 + 2.71050543121376e-20*x**3 - 2.77555756156289e-17*x**2*y**3 + 0.148747545665497*x**2*y**2 - 0.00260199788336145*x**2*y + 1.13789994899189e-5*x**2 + 1.11022302462516e-16*x*y**4 - 0.333194502290712*x*y**3 + 0.00874271288809449*x*y**2 - 0.00305141778988219*x*y - 0.000398750073281879*x + 1.0*y**5 + 6.93889390390723e-18*y**4 - 0.000764987377708268*y**3 + 0.000429874831263535*y**2 + 5.21256433395786e-5*y - 4.88333641416651e-7, x, y, domain='RR') Poly(0.714285714285714*x + 0.043731778425656, x, domain='RR')
Poly(1.73472347597681e-18*x**4*y - 4.33680868994202e-19*x**4 + 2.71050543121376e-20*x**3 - 2.77555756156289e-17*x**2*y**3 + 0.148747545665497*x**2*y**2 - 0.00260199788336145*x**2*y + 1.13789994899189e-5*x**2 + 1.11022302462516e-16*x*y**4 - 0.333194502290712*x*y**3 + 0.00874271288809449*x*y**2 - 0.00305141778988219*x*y - 0.000398750073281879*x + 1.0*y**5 + 6.93889390390723e-18*y**4 - 0.000764987377708268*y**3 + 0.0004298748312635

looks like urgetter is working fine too.

## Hensel's Lemma and its auxiliar functions.

In [81]:
...

Ellipsis

## The algorithm: solving limits of the form f/g with f and g polynomials in two variables

In [ ]:
def quotientLimit(f,g):
    '''
    This function finds the limit f/g as (x,y) tends to (0,0).
    '''
    x = sympy.Symbol('x')
    y = sympy.Symbol('y')
    h = hgetter(f,g)
    h = monicMaker(h)
    modified_h, phi_inverser = phiAutomorphism(h)
    (r, u_r) = urgetter(modified_h)
    modified_h = newtonAutomorphism(modified_h, r, u_r)
    
    d = sympy.degree(h, y)
    
    poly_accumulator = 1
    F = sympy.simplify(modified_h/x**(d*u_r)).as_poly()
    